# Martha's Vineyard Snowflake Tutorial

This notebook demonstrates Snowflake fundamentals including:
- Creating databases and schemas
- Creating tables
- Setting up stages for file uploads
- Loading data from CSV files into tables

**Scenario**: Build a wine production database for Martha's Vineyard tracking grape types and annual yield data.


## Step 1: Create Database, Schema, Tables, and Stage

Execute the following SQL commands to set up your Snowflake objects. You can run these directly in Snowflake UI or execute them from this notebook.


In [ ]:
-- Create database
CREATE DATABASE IF NOT EXISTS MarthasVineyard;

-- Use database
USE DATABASE MarthasVineyard;

-- Create schema
CREATE SCHEMA IF NOT EXISTS MyVineyard;

-- Use schema
USE SCHEMA MyVineyard;

-- Create table for grape types
CREATE TABLE IF NOT EXISTS grape_types (
    grape_id INT PRIMARY KEY,
    grape_name VARCHAR(100) NOT NULL,
    grape_type VARCHAR(50) NOT NULL,
    description VARCHAR(500)
);

-- Create table for annual yield and financial data
CREATE TABLE IF NOT EXISTS annual_yield (
    year INT NOT NULL,
    grape_id INT NOT NULL,
    yield_tons DECIMAL(10, 2) NOT NULL,
    production_cost DECIMAL(12, 2) NOT NULL,
    earnings DECIMAL(12, 2) NOT NULL,
    profit DECIMAL(12, 2),
    PRIMARY KEY (year, grape_id),
    FOREIGN KEY (grape_id) REFERENCES grape_types(grape_id)
);

-- Create internal stage
CREATE STAGE IF NOT EXISTS vineyard_stage
DIRECTORY = (ENABLE = true);


## Step 2: Upload CSV Files to Stage (Using Snowflake UI)

Before proceeding, upload the two CSV files to the `vineyard_stage` using the Snowflake UI:

1. Navigate to Data > Databases > MarthasVineyard > Schemas > MyVineyard > Stages > vineyard_stage
2. Upload `grape_types.csv` 
3. Upload `annual_yield.csv`

Alternatively, you can use the Snowflake CLI or execute PUT commands directly.


## Step 3: Load Data from Stage into Tables

Execute the COPY INTO commands to load the CSV files from the stage into the tables.


In [ ]:
-- Load data into grape_types table
COPY INTO grape_types
FROM @vineyard_stage/grape_types.csv
FILE_FORMAT = (TYPE = CSV SKIP_HEADER = 1 FIELD_DELIMITER = ',' NULL_IF = ('NULL') EMPTY_FIELD_AS_NULL = true)
ON_ERROR = SKIP_FILE;


## Step 4: Load Data into Annual Yield Table


In [ ]:
-- Load data into annual_yield table
COPY INTO annual_yield (year, grape_id, yield_tons, production_cost, earnings, profit)
FROM @vineyard_stage/annual_yield.csv
FILE_FORMAT = (TYPE = CSV SKIP_HEADER = 1 FIELD_DELIMITER = ',' NULL_IF = ('NULL') EMPTY_FIELD_AS_NULL = true)
ON_ERROR = SKIP_FILE;


## Step 5: Establish Connection and Verify Data

Now we'll establish a connection using Snowflake's get_active_connection() and verify the data was loaded correctly.


In [ ]:
from snowflake.connector import get_active_connection
import pandas as pd
import matplotlib.pyplot as plt

# Get active connection
conn = get_active_connection()
cursor = conn.cursor()

print("Connected to Snowflake successfully!")


## Step 6: Verify Data Load


In [ ]:
# Query grape_types table
query_grapes = "SELECT * FROM MarthasVineyard.MyVineyard.grape_types;"
df_grapes = pd.read_sql(query_grapes, conn)
print("\n=== Grape Types Data ===")
print(df_grapes)
print(f"\nTotal grape types: {len(df_grapes)}")


## Step 7: View Annual Yield Data


In [ ]:
# Query annual_yield table
query_yield = "SELECT * FROM MarthasVineyard.MyVineyard.annual_yield ORDER BY year, grape_type;"
df_yield = pd.read_sql(query_yield, conn)
print("\n=== Annual Yield Data ===")
print(df_yield)
print(f"\nTotal records: {len(df_yield)}")


## Step 8: Graph Five-Year Yield by Grape Variety

Create a visualization showing the yield trends for each grape variety over the 5-year period.


In [ ]:
# Query data for visualization
query_yield_summary = """
SELECT ay.year, gt.grape_name, ay.yield_tons 
FROM MarthasVineyard.MyVineyard.annual_yield ay
JOIN MarthasVineyard.MyVineyard.grape_types gt ON ay.grape_id = gt.grape_id
ORDER BY ay.year, gt.grape_name
"""
df_pivot = pd.read_sql(query_yield_summary, conn)

# Pivot data for plotting
yield_pivot = df_pivot.pivot(index='YEAR', columns='GRAPE_NAME', values='YIELD_TONS')

# Create figure and plot
plt.figure(figsize=(12, 6))
for grape_type in yield_pivot.columns:
    plt.plot(yield_pivot.index, yield_pivot[grape_type], marker='o', label=grape_type, linewidth=2)

plt.xlabel('Year', fontsize=12)
plt.ylabel('Yield (Tons)', fontsize=12)
plt.title("Martha's Vineyard: Five-Year Yield by Grape Variety", fontsize=14, fontweight='bold')
plt.legend(title='Grape Type', fontsize=10)
plt.grid(True, alpha=0.3)
plt.xticks(yield_pivot.index)
plt.tight_layout()
plt.show()

print("\n=== Yield Summary by Grape Type ===")
print(yield_pivot)


In [ ]:
# Additional analysis: Profitability trend
query_profit = """
SELECT ay.year, gt.grape_name, ay.profit 
FROM MarthasVineyard.MyVineyard.annual_yield ay
JOIN MarthasVineyard.MyVineyard.grape_types gt ON ay.grape_id = gt.grape_id
ORDER BY ay.year, gt.grape_name
"""
df_profit = pd.read_sql(query_profit, conn)
profit_pivot = df_profit.pivot(index='YEAR', columns='GRAPE_NAME', values='PROFIT')

# Plot profitability
plt.figure(figsize=(12, 6))
for grape_type in profit_pivot.columns:
    plt.plot(profit_pivot.index, profit_pivot[grape_type], marker='s', label=grape_type, linewidth=2)

plt.xlabel('Year', fontsize=12)
plt.ylabel('Profit ($)', fontsize=12)
plt.title("Martha's Vineyard: Five-Year Profit by Grape Variety", fontsize=14, fontweight='bold')
plt.legend(title='Grape Type', fontsize=10)
plt.grid(True, alpha=0.3)
plt.xticks(profit_pivot.index)
plt.ticklabel_format(style='plain', axis='y')
plt.tight_layout()
plt.show()

print("\n=== Profit Summary by Grape Type ===")
print(profit_pivot)


## Step 9: Revenue Analysis

Analyze total revenue and profitability by grape type over the five-year period.


In [ ]:
# Total revenue by grape type
query_revenue = """
SELECT 
    gt.grape_name,
    SUM(ay.earnings) as total_earnings,
    SUM(ay.production_cost) as total_cost,
    SUM(ay.profit) as total_profit
FROM MarthasVineyard.MyVineyard.annual_yield ay
JOIN MarthasVineyard.MyVineyard.grape_types gt ON ay.grape_id = gt.grape_id
GROUP BY gt.grape_name
ORDER BY total_profit DESC
"""

df_revenue = pd.read_sql(query_revenue, conn)
print("\n=== Revenue Analysis by Grape Type (2020-2024) ===")
print(df_revenue.to_string(index=False))

# Visualization
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

# Earnings comparison
ax1.bar(df_revenue['GRAPE_NAME'], df_revenue['TOTAL_EARNINGS'], color='green', alpha=0.7)
ax1.set_xlabel('Grape Type', fontsize=11)
ax1.set_ylabel('Total Earnings ($)', fontsize=11)
ax1.set_title('Total Earnings by Grape Type', fontsize=12, fontweight='bold')
ax1.ticklabel_format(style='plain', axis='y')

# Profit comparison
ax2.bar(df_revenue['GRAPE_NAME'], df_revenue['TOTAL_PROFIT'], color='darkgreen', alpha=0.7)
ax2.set_xlabel('Grape Type', fontsize=11)
ax2.set_ylabel('Total Profit ($)', fontsize=11)
ax2.set_title('Total Profit by Grape Type', fontsize=12, fontweight='bold')
ax2.ticklabel_format(style='plain', axis='y')

plt.tight_layout()
plt.show()


In [ ]:
# Close connection
cursor.close()
conn.close()
print("Snowflake connection closed.")


## Optional: Additional Cleanup Commands

Uncomment the SQL below if you want to drop tables or the entire database.


In [ ]:
-- Optional: Drop tables
-- DROP TABLE IF EXISTS MarthasVineyard.MyVineyard.grape_types;
-- DROP TABLE IF EXISTS MarthasVineyard.MyVineyard.annual_yield;
-- DROP STAGE IF EXISTS MarthasVineyard.MyVineyard.vineyard_stage;


In [ ]:
-- Optional: Drop schema
-- DROP SCHEMA IF EXISTS MarthasVineyard.MyVineyard;

-- Optional: Drop database
-- DROP DATABASE IF EXISTS MarthasVineyard;
